In [1]:
import os
import tqdm
import wandb
import warnings
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import concurrent.futures

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging
warnings.filterwarnings('ignore', category=UserWarning, module='google.protobuf')

from matplotlib.axes import Axes
from wandb.apis.public import Run

from typing import Union, List, Dict
from src.visualization import set_themes

set_themes() # Set custom themes for plots
pl.Config.set_tbl_rows(20) # Set Polars table display rows limit

pd.set_option('future.no_silent_downcasting', True)

failed to send, dropping 5 traces to intake at http://localhost:8126/v0.5/traces after 3 retries


In [2]:
cache_file = "wandb/summary.parquet"
config = {
    "model": "matrix_factorization",
    "ensure_available_locally": False
}
sorting_criterion = {
    "epoch/test_hitrate@50": 0.5,
    "epoch/test_ndcg@50": 0.25,
}

if cache_file is not None and os.path.exists(cache_file):
    print(f"Loading cached experiment runs from {cache_file}...")
    experiment_runs = pl.read_parquet(cache_file)
    print(f"Loaded {len(experiment_runs)} runs from cache.")
else:
    print("No cache file found. Fetching experiment runs from Weights & Biases...")
    api = wandb.Api() # Initialize Weights & Biases API, used for fetching run data

    def fetch_run_metadata(run: Run, considered_metrics: Union[str, Dict[str, float]] = "epoch/epoch") -> Dict:
        run_config = {}
        for key, value in run.config.items():
            # Convert lists and dicts to strings
            if isinstance(value, (list, dict)):
                run_config[key] = str(value)
            else:
                run_config[key] = value

        run_history = run.history()
        run_history = run_history.replace({"Infinity": np.inf, "NaN": np.nan})

        if isinstance(considered_metrics, str):
            run_history["score"] = run_history[considered_metrics]
        elif isinstance(considered_metrics, dict):
            run_history["score"] = sum(
                run_history[metric] * weight for metric, weight in considered_metrics.items()
            )
        else:
            raise ValueError("considered_metrics must be either a string or a dictionary")
        
        best_summary = run_history.iloc[run_history["score"].argmax()]
        best_summary = {f"best:{key}": val for key, val in best_summary.items()}
        
        return {
            "run_id": run.id,
            "run_name": run.name,
            "sweep_id": run.sweep.id if run.sweep else None,
            "model": run.config.get("model"),
            **run_config,
            **{metric: run_history[metric].to_list() for metric in run_history},
            **best_summary,
            "gpu_type": run.metadata.get("gpu"),
            "cpu_count": run.metadata.get("cpu_count"),
        }

    batch_size = 16
    records = []
    futures = {}
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=batch_size)
    runs:List[Run] = api.runs("feedr/peppermint-matrix", per_page=2*batch_size-1, filters={"config.model": config["model"]})
    run_iterator = iter(runs)
    with tqdm.tqdm(total=len(runs), ncols=128) as pbar:
        while len(records) < len(runs):
            # submit new tasks if we empty slots in the batch
            while len(futures) < batch_size and len(records) + len(futures) < len(runs):
                current_runs = next(run_iterator)
                current_future = executor.submit(fetch_run_metadata, current_runs, sorting_criterion)
                futures[current_future] = current_runs

            # check for completed tasks
            finished_futures, _ = concurrent.futures.wait(futures.keys(), return_when=concurrent.futures.FIRST_COMPLETED, timeout=0.1)
            for finished_future in finished_futures:
                finished_run = futures.pop(finished_future)
                records.append(finished_future.result())
                pbar.update(1)

    # Create a Polars DataFrame from the records
    experiment_runs = pl.DataFrame(records, infer_schema_length=None)
    
# Tag run as available locally if the model files exist
local_run_ids = []
local_sweep_ids = os.listdir(f"./models/{config['model']}/")
for sweep_id in local_sweep_ids:
    local_run_ids.extend([run_id for run_id in os.listdir(f"./models/{config['model']}/{sweep_id}/")])
    
experiment_runs = experiment_runs.with_columns(
    available_locally=pl.col("run_id").is_in(local_run_ids)
)

if config["ensure_available_locally"]:
    experiment_runs = experiment_runs.filter(pl.col("available_locally") == True)

experiment_runs = experiment_runs.sort("_timestamp", descending=False)
experiment_runs = experiment_runs.with_columns(
    run_duration_second=pl.col("_runtime").list.max(),
    run_duration_minute=(pl.col("_runtime").list.max() / 60)
)
experiment_runs.select(
    pl.col("run_id"),
    pl.col("run_name"),
    pl.col("sweep_id"),
    pl.col("model"),
    pl.col("embedding_dimension"),
    pl.col("shuffle"),
    pl.col("best:epoch/epoch"),
    pl.col("best:epoch/train_loss"),
    pl.col("best:epoch/test_loss"),
    pl.col("best:epoch/test_recall@10"),
    pl.col("best:epoch/test_ndcg@10"),
)

Loading cached experiment runs from wandb/summary.parquet...
Loaded 256 runs from cache.


run_id,run_name,sweep_id,model,embedding_dimension,shuffle,best:epoch/epoch,best:epoch/train_loss,best:epoch/test_loss,best:epoch/test_recall@10,best:epoch/test_ndcg@10
str,str,str,str,i64,bool,f64,f64,f64,f64,f64
"""o94q0juk""","""logical-sweep-1""","""nbysw136""","""matrix_factorization""",256,false,52.0,0.330564,0.372714,0.026077,0.100539
"""4ftaae0p""","""stilted-sweep-3""","""nbysw136""","""matrix_factorization""",4,false,59.0,0.693148,0.693148,0.006594,0.02556
"""fway5u2z""","""breezy-sweep-4""","""nbysw136""","""matrix_factorization""",512,false,4.0,0.693147,0.693147,0.0026,0.012059
"""bphcl2xf""","""clean-sweep-2""","""nbysw136""","""matrix_factorization""",1024,false,1.0,0.237272,0.239171,0.024744,0.095121
"""fftz1dek""","""trim-sweep-5""","""nbysw136""","""matrix_factorization""",256,true,57.0,0.319744,0.366027,0.025562,0.099834
"""otb8suw9""","""scarlet-sweep-6""","""nbysw136""","""matrix_factorization""",4,true,63.0,0.133806,0.217083,0.021467,0.082659
"""lvre7srl""","""solar-sweep-7""","""nbysw136""","""matrix_factorization""",256,true,8.0,0.036125,0.194509,0.021456,0.084617
"""dcbj92eg""","""ruby-sweep-8""","""nbysw136""","""matrix_factorization""",256,false,3.0,0.078173,0.179263,0.024295,0.094158
"""x17mnyw8""","""breezy-sweep-9""","""nbysw136""","""matrix_factorization""",8,false,61.0,0.111218,0.207902,0.023699,0.089898


# Parameter Comparison

## Embedding Dimension vs Regularization

In [3]:
experiment_summary = experiment_runs.group_by("embedding_dimension", "l2_regularization").agg(
    pl.col("run_id").count().alias("num_runs"),
    pl.col("best:epoch/epoch").mean(),
    pl.col("best:epoch/test_recall@10").mean(),
    pl.col("best:epoch/test_ndcg@10").mean(),
    pl.col("best:epoch/test_recall@20").mean(),
    pl.col("best:epoch/test_ndcg@20").mean(),
).sort("embedding_dimension", "l2_regularization")
experiment_summary

embedding_dimension,l2_regularization,num_runs,best:epoch/epoch,best:epoch/test_recall@10,best:epoch/test_ndcg@10,best:epoch/test_recall@20,best:epoch/test_ndcg@20
i64,f64,u32,f64,f64,f64,f64,f64
2,0.0,4,41.0,0.014936,0.058054,0.026526,0.07701
2,1.0000e-8,2,56.5,0.015958,0.063231,0.028426,0.083195
2,0.0000001,1,51.0,0.015485,0.059923,0.027484,0.079607
2,0.000001,1,61.0,0.014238,0.057333,0.026154,0.076683
2,0.00001,3,52.0,0.005195,0.023738,0.009016,0.030197
2,0.0001,5,10.6,0.002627,0.011328,0.004475,0.01525
2,0.001,3,39.333333,0.000568,0.002992,0.001017,0.004111
2,0.01,4,9.25,0.000492,0.002238,0.001114,0.003655
4,0.0,3,48.333333,0.020096,0.077747,0.035029,0.100483


In [4]:
experiment_summary[["embedding_dimension", "l2_regularization", "num_runs"]].pivot(
    values=["num_runs"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_939398/2842099446.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "num_runs"]].pivot(


embedding_dimension,0.0,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,u32,u32,u32,u32,u32,u32,u32,u32
2,4,2,1,1,3,5,3,4
4,3,2,3,3,4,6,2,3
8,2,3,3,2,4,6,3,3
16,4,6,2,null,5,5,6,3
32,4,2,2,8,3,3,5,4
64,4,3,3,5,2,2,4,3
128,5,null,6,3,2,3,4,1
256,5,6,4,4,2,1,3,5
512,1,3,null,3,1,4,2,2


In [5]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/epoch"]].pivot(
    values=["best:epoch/epoch"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_939398/3529959983.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/epoch"]].pivot(


embedding_dimension,0.0,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,f64,f64,f64,f64,f64,f64,f64,f64
2,41.0,56.5,51.0,61.0,52.0,10.6,39.333333,9.25
4,48.333333,60.0,62.0,56.666667,40.25,9.166667,54.0,8.666667
8,57.5,56.666667,61.0,58.0,54.5,9.333333,42.0,9.333333
16,36.0,46.166667,62.5,null,39.6,7.2,62.0,9.666667
32,16.0,48.0,62.0,55.125,60.333333,9.333333,57.0,8.5
64,11.5,17.333333,58.333333,54.8,53.5,12.0,62.0,8.0
128,13.2,null,54.833333,51.666667,56.5,7.333333,57.5,9.0
256,7.2,7.0,51.5,56.5,62.0,12.0,61.666667,8.0
512,1.0,4.666667,null,60.333333,63.0,5.5,62.5,8.5


In [6]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_recall@20"]].pivot(
    values=["best:epoch/test_recall@20"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_939398/128151351.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_recall@20"]].pivot(


embedding_dimension,0.0,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,f64,f64,f64,f64,f64,f64,f64,f64
2,0.026526,0.028426,0.027484,0.026154,0.009016,0.004475,0.001017,0.001114
4,0.035029,0.035989,0.036743,0.036439,0.016877,0.004478,0.001067,0.001341
8,0.039197,0.039903,0.042342,0.041993,0.017602,0.004518,0.000781,0.00134
16,0.040109,0.041057,0.046312,null,0.023459,0.004471,0.001178,0.001179
32,0.040013,0.040648,0.048136,0.044624,0.015782,0.004726,0.000946,0.001464
64,0.039624,0.041139,0.04855,0.045224,0.018902,0.004865,0.001109,0.001852
128,0.038498,null,0.048648,0.044754,0.01724,0.004553,0.00072,0.001337
256,0.037937,0.041123,0.047656,0.045245,0.013806,0.004868,0.001104,0.001728
512,0.040351,0.040249,null,0.044987,0.014279,0.00429,0.00075,0.001676


In [7]:
experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_ndcg@20"]].pivot(
    values=["best:epoch/test_ndcg@20"],
    index="embedding_dimension",
    columns="l2_regularization"
)

/tmp/ipykernel_939398/3686121945.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "l2_regularization", "best:epoch/test_ndcg@20"]].pivot(


embedding_dimension,0.0,1e-8,1e-7,1e-6,0.00001,0.0001,0.001,0.01
i64,f64,f64,f64,f64,f64,f64,f64,f64
2,0.07701,0.083195,0.079607,0.076683,0.030197,0.01525,0.004111,0.003655
4,0.100483,0.102319,0.103656,0.103239,0.050983,0.01593,0.004144,0.004044
8,0.11097,0.112247,0.117917,0.118006,0.053196,0.015953,0.002973,0.004392
16,0.113707,0.115484,0.12745,null,0.070878,0.016023,0.004485,0.003583
32,0.114032,0.115495,0.133207,0.125483,0.048879,0.016388,0.00361,0.004424
64,0.112689,0.115975,0.134344,0.126898,0.05758,0.01695,0.00426,0.005815
128,0.11005,null,0.134197,0.125806,0.053038,0.016039,0.002773,0.004119
256,0.107735,0.116549,0.131321,0.126012,0.042399,0.017381,0.004054,0.004914
512,0.112442,0.112616,null,0.125612,0.04428,0.01541,0.002894,0.005035


# Cross-GPU Training

In [15]:
experiment_runs.group_by("embedding_dimension").agg(
    pl.col("run_duration_minute").mean()
).sort("embedding_dimension")

embedding_dimension,run_duration_minute
i64,f64
2,22.742412
4,22.665075
8,23.885482
16,22.985601
32,23.285895
64,21.641968
128,23.73886
256,24.837704
512,25.84378


In [16]:
experiment_runs.group_by("gpu_type").agg(
    pl.col("run_duration_minute").mean()
).sort("gpu_type")

gpu_type,run_duration_minute
str,f64
"""NVIDIA A100-SXM4-40GB""",47.189427
"""NVIDIA A10G""",23.009375
"""NVIDIA L4""",28.125109


failed to send, dropping 2 traces to intake at http://localhost:8126/v0.5/traces after 3 retries, 6 additional messages skipped


In [17]:
experiment_summary = experiment_runs.filter(pl.col("l2_regularization") == 0.).group_by("embedding_dimension", "gpu_type").agg(
    pl.col("run_id").count().alias("num_runs"),
    pl.col("best:epoch/epoch").mean(),
    pl.col("best:epoch/test_recall@10").mean(),
    pl.col("best:epoch/test_ndcg@10").mean(),
    pl.col("best:epoch/test_recall@50").mean(),
    pl.col("best:epoch/test_ndcg@50").mean(),
).sort("embedding_dimension", "gpu_type")
experiment_summary

embedding_dimension,gpu_type,num_runs,best:epoch/epoch,best:epoch/test_recall@10,best:epoch/test_ndcg@10,best:epoch/test_recall@50,best:epoch/test_ndcg@50
i64,str,u32,f64,f64,f64,f64,f64
2,"""NVIDIA A100-SXM4-40GB""",1,63.0,0.014696,0.059235,0.05268,0.105463
2,"""NVIDIA A10G""",19,63.0,0.014549,0.056882,0.053937,0.103935
2,"""NVIDIA L4""",1,63.0,0.014849,0.058681,0.056454,0.10793
4,"""NVIDIA A100-SXM4-40GB""",1,63.0,0.019013,0.074248,0.070179,0.129829
4,"""NVIDIA A10G""",14,63.0,0.019431,0.075468,0.069966,0.130528
4,"""NVIDIA L4""",4,63.0,0.019367,0.075652,0.069782,0.130409
8,"""NVIDIA A100-SXM4-40GB""",2,63.0,0.02136,0.082498,0.078069,0.141937
8,"""NVIDIA A10G""",19,63.0,0.021914,0.084904,0.078703,0.143965
8,"""NVIDIA L4""",3,63.0,0.021715,0.083619,0.078856,0.143085


In [18]:
experiment_summary[["embedding_dimension", "gpu_type", "num_runs"]].pivot(
    values=["num_runs"],
    index="embedding_dimension",
    columns="gpu_type"
)

/tmp/ipykernel_900224/1667364229.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "gpu_type", "num_runs"]].pivot(


embedding_dimension,NVIDIA A100-SXM4-40GB,NVIDIA A10G,NVIDIA L4
i64,u32,u32,u32
2,1,19,1
4,1,14,4
8,2,19,3
16,1,27,1
32,null,27,3
64,1,21,2
128,null,26,null
256,3,23,4
512,2,20,3


In [19]:
experiment_summary[["embedding_dimension", "gpu_type", "best:epoch/test_recall@50"]].pivot(
    values=["best:epoch/test_recall@50"],
    index="embedding_dimension",
    columns="gpu_type"
)

/tmp/ipykernel_900224/93591780.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "gpu_type", "best:epoch/test_recall@50"]].pivot(


embedding_dimension,NVIDIA A100-SXM4-40GB,NVIDIA A10G,NVIDIA L4
i64,f64,f64,f64
2,0.05268,0.053937,0.056454
4,0.070179,0.069966,0.069782
8,0.078069,0.078703,0.078856
16,0.079871,0.079915,0.081216
32,null,0.077398,0.077563
64,0.07268,0.073611,0.072878
128,null,0.069959,null
256,0.066112,0.066402,0.066835
512,0.062456,0.063286,0.063337


In [20]:
experiment_summary[["embedding_dimension", "gpu_type", "best:epoch/test_ndcg@50"]].pivot(
    values=["best:epoch/test_ndcg@50"],
    index="embedding_dimension",
    columns="gpu_type"
)

/tmp/ipykernel_900224/2601597168.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "gpu_type", "best:epoch/test_ndcg@50"]].pivot(


embedding_dimension,NVIDIA A100-SXM4-40GB,NVIDIA A10G,NVIDIA L4
i64,f64,f64,f64
2,0.105463,0.103935,0.10793
4,0.129829,0.130528,0.130409
8,0.141937,0.143965,0.143085
16,0.146282,0.145748,0.147393
32,null,0.142054,0.142608
64,0.136999,0.136925,0.135871
128,null,0.132737,null
256,0.127316,0.128326,0.128494
512,0.123614,0.12421,0.124616
